In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("E:/Documents/sonar Dataset.csv")
df.head()

Column1  Column2  Column3  Column4  Column5  Column6  Column7  Column8  \
0   0.0200   0.0371   0.0428   0.0207   0.0954   0.0986   0.1539   0.1601   
1   0.0453   0.0523   0.0843   0.0689   0.1183   0.2583   0.2156   0.3481   
2   0.0262   0.0582   0.1099   0.1083   0.0974   0.2280   0.2431   0.3771   
3   0.0100   0.0171   0.0623   0.0205   0.0205   0.0368   0.1098   0.1276   
4   0.0762   0.0666   0.0481   0.0394   0.0590   0.0649   0.1209   0.2467   

   Column9  Column10  ...  Column52  Column53  Column54  Column55  Column56  \
0   0.3109    0.2111  ...    0.0027    0.0065    0.0159    0.0072    0.0167   
1   0.3337    0.2872  ...    0.0084    0.0089    0.0048    0.0094    0.0191   
2   0.5598    0.6194  ...    0.0232    0.0166    0.0095    0.0180    0.0244   
3   0.0598    0.1264  ...    0.0121    0.0036    0.0150    0.0085    0.0073   
4   0.3564    0.4459  ...    0.0031    0.0054    0.0105    0.0110    0.0015   

   Column57  Column58  Column59  Column60  Column61  
0    0.0180    0.0084    0.0090    0.0032         R  
1    0.0140    0.0049    0.0052    0.0044         R  
2    0.0316    0.0164    0.0095    0.0078         R  
3    0.0050    0.0044    0.0040    0.0117         R  
4    0.0072    0.0048    0.0107    0.0094         R  

[5 rows x 61 columns]

In [3]:
x=df.iloc[:,:-1]
y=df.iloc[:,-1]

In [4]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y=le.fit_transform(y)

In [5]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,stratify=y,shuffle=True,random_state=148)

In [6]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_train_sc=scaler.fit_transform(x_train)
x_test_sc=scaler.fit_transform(x_test)

In [7]:
from sklearn.preprocessing import MinMaxScaler
scaler= MinMaxScaler()
x_train_mm = scaler.fit_transform(x_train)
x_test_mm = scaler.fit_transform(x_test)

In [11]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import  GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import warnings
warnings.simplefilter('ignore')

## Logistic Regression

In [12]:
lr_parameters_grid = {
    'penalty': ['l1', 'l2', 'elasticnet'],
    'C': [0.1, 0.5, 1, 10, 100],
    'solver': ['sag', 'saga', 'liblinear','lbfgs'],
    'l1_ratio': [0.25, 0.5, 0.75],
    'max_iter': [50,100,250,400],
}

lr = LogisticRegression(class_weight='balanced')
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=148)
lr_grid_cv = GridSearchCV(lr, lr_parameters_grid, scoring='roc_auc', cv=cv, verbose=0).fit(x_train,y_train)

lr_best_params = lr_grid_cv.best_params_
lr_best_est = lr_grid_cv.best_estimator_
lr_p = lr_best_est.predict(x_test)
print(accuracy_score(y_test,lr_p))
print('Best Param: ',lr_best_params)
print('Best_Estimator: ',lr_best_est)

0.7857142857142857
Best Param:  {'C': 1, 'l1_ratio': 0.75, 'max_iter': 50, 'penalty': 'elasticnet', 'solver': 'saga'}
Best_Estimator:  LogisticRegression(C=1, class_weight='balanced', l1_ratio=0.75, max_iter=50,
                   penalty='elasticnet', solver='saga')


In [57]:
lr_best_params={'C': 1,
 'l1_ratio': 0.75,
 'max_iter': 250,
 'penalty': 'elasticnet',
 'solver': 'saga'}

lr = LogisticRegression(**lr_best_params, class_weight='balanced').fit(x_train_sc,y_train)
lr_p = lr.predict(x_test_sc)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,lr_p))

0.8095238095238095


## KNN Classifier

In [11]:
from math import sqrt
sqrt(len(df)),sqrt(len(x_test)),sqrt(len(x_train))

(14.422205101855956, 6.48074069840786, 12.884098726725126)

In [12]:
knn_parameters_grid = {
    'n_neighbors': [5,7,11,13,15],
    'algorithm': ['auto','ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [20, 30, 40],
    'p': [1, 2]
}

knn = KNeighborsClassifier()
knn_grid_cv = GridSearchCV(knn, knn_parameters_grid, scoring='roc_auc', cv=cv, verbose=0).fit(x_train,y_train)
knn_best_params = knn_grid_cv.best_params_
knn_best_params

{'algorithm': 'auto', 'leaf_size': 20, 'n_neighbors': 5, 'p': 1}

In [13]:
knn_best_params = {'algorithm': 'ball_tree', 'leaf_size': 20, 'n_neighbors': 3, 'p': 1}

knn = KNeighborsClassifier(**knn_best_params).fit(x_train,y_train)
knn_p = knn.predict(x_test)
print(accuracy_score(y_test,knn_p))

0.9047619047619048


## Random Forest Classifier

In [59]:
rf_parameters_grid = {
    'n_estimators': [200, 500],
    'criterion':['gini','entropy','log_loss'],
    'max_features': ['sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

rf = RandomForestClassifier()
rf_grid_cv = GridSearchCV(rf, rf_parameters_grid, scoring='roc_auc', cv=cv, verbose=0).fit(x_train,y_train)
rf_best_params = rf_grid_cv.best_params_
rf_best_est = rf_grid_cv.best_estimator_
rf_p = rf_best_est.predict(x_test)
print(accuracy_score(y_test,rf_p))
print('Best Param: ',rf_best_params)
print('Best_Estimator: ',rf_best_est)

0.8095238095238095
Best Param:  {'criterion': 'gini', 'max_depth': 8, 'max_features': 'log2', 'n_estimators': 500}
Best_Estimator:  RandomForestClassifier(max_depth=8, max_features='log2', n_estimators=500)


In [64]:
rf_best_params = {'criterion': 'entropy','max_features':'log2','max_depth': 8,'n_estimators': 200,'random_state':148}
rf = RandomForestClassifier(**rf_best_params).fit(x_train,y_train)
rf_p = rf.predict(x_test)
print(accuracy_score(y_test,rf_p))

0.8571428571428571


## SVC

In [47]:
svc_parameters_grid = {
    'C': [1.0,1.5,2.0,3.0,4.0],
    'kernel':['linear','poly','rbf','sigmoid'],
    'degree': [2,3,4],
    'gamma':['scale','auto'],
    'decision_function_shape' : ['ovr','ovo']
}
from sklearn.svm import SVC

svc = SVC()
svc_grid_cv = GridSearchCV(svc, svc_parameters_grid, scoring='roc_auc', cv=cv, verbose=0).fit(x_train,y_train)
svc_best_est = svc_grid_cv.best_estimator_
svc_best_param = svc_grid_cv.best_params_
svc_p = svc_best_est.predict(x_test)
print(accuracy_score(y_test,svc_p))
print('Best Param: ',svc_best_param)
print('Best_Estimator: ',svc_best_est)

0.9047619047619048
Best Param:  {'C': 4.0, 'decision_function_shape': 'ovr', 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}
Best_Estimator:  SVC(C=4.0, degree=2)


In [50]:
best_param = {'C': 4.0, 'decision_function_shape': 'ovr', 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}
svc= SVC(**best_param).fit(x_train,y_train)
svc_p = svc.predict(x_test)
print(accuracy_score(y_test,svc_p))

0.9047619047619048


## SGD Classifier

In [66]:
sgd_parameters_grid = {
    'loss': ['hinge', 'log_loss', 'modified_huber','squared_hinge', 'perceptron'],
    'penalty':['l1','l2','elasticnet'],
    'alpha': [0.01,0.001,0.0001],
    'class_weight':['balanced']
}
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier()
sgd_grid_cv = GridSearchCV(sgd, sgd_parameters_grid, scoring='roc_auc', cv=cv, verbose=0).fit(x_train,y_train)
sgd_best_est = sgd_grid_cv.best_estimator_
sgd_best_param = sgd_grid_cv.best_params_
sgd_p = sgd_best_est.predict(x_test)
print(accuracy_score(y_test,sgd_p))
print('Best Param: ',sgd_best_param)
print('Best_Estimator: ',sgd_best_est)

0.7857142857142857
Best Param:  {'alpha': 0.01, 'class_weight': 'balanced', 'loss': 'hinge', 'penalty': 'elasticnet'}
Best_Estimator:  SGDClassifier(alpha=0.01, class_weight='balanced', penalty='elasticnet')


In [71]:
best_param = {'alpha': 0.01, 'class_weight': 'balanced', 'loss': 'hinge', 'penalty': 'elasticnet'}
sgd= SGDClassifier(**best_param).fit(x_train,y_train)
sgd_p = sgd.predict(x_test)
print(accuracy_score(y_test,sgd_p))

0.7857142857142857


## Quadratic Discriminant Analysis

In [14]:
param_grid = {
    'reg_param':[0.0,0.1,0.5,1.0,10.0]
}

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
qda = QuadraticDiscriminantAnalysis()

qda_grid_cv = GridSearchCV(qda, param_grid, scoring='roc_auc', cv=cv, verbose=0).fit(x_train,y_train)
qda_best_est = qda_grid_cv.best_estimator_
qda_best_param = qda_grid_cv.best_params_
qda_p = qda_best_est.predict(x_test)
print(accuracy_score(y_test,qda_p))
print('Best Param: ',qda_best_param)
print('Best_Estimator: ',qda_best_est)

0.8809523809523809
Best Param:  {'reg_param': 0.1}
Best_Estimator:  QuadraticDiscriminantAnalysis(reg_param=0.1)


In [15]:
best_param = {'reg_param': 0.1}
qda = QuadraticDiscriminantAnalysis(**best_param).fit(x_train,y_train)
qda_p = qda.predict(x_test)
print(accuracy_score(y_test,qda_p))

0.8809523809523809


## XGBoost Classifier

In [18]:
xgb = XGBClassifier(n_estimators=100).fit(x_train,y_train)
xgb_p =xgb.predict(x_test)
print(accuracy_score(y_test,xgb_p))

0.8571428571428571


## MLP Classifier

In [16]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV

mlp = MLPClassifier().fit(x_train,y_train)
mlp_p = mlp.predict(x_test)
print(accuracy_score(y_test,mlp_p))

0.8095238095238095


In [20]:
parameters = {'hidden_layer_sizes':[50, 60, 100, 200],
              'activation':['identity','logistic','tanh','relu'],
              'solver':['lbfgs','sgd','adam'],
              'alpha': [0.001, 0.01, 0.1], 
              'max_iter': [200, 500, 800], 
              'learning_rate_init':[0.0001, 0.001, 0.01, 0.1]}


model = MLPClassifier()
clf = RandomizedSearchCV(estimator=model, param_distributions=parameters, cv=cv).fit(x_train,y_train)
print(clf.best_params_)
best_model = clf.best_estimator_
clf_p= best_model.predict(x_test)
print(accuracy_score(y_test,clf_p))

{'solver': 'lbfgs', 'max_iter': 800, 'learning_rate_init': 0.0001, 'hidden_layer_sizes': 100, 'alpha': 0.1, 'activation': 'tanh'}
0.8809523809523809


In [37]:
#best_params={'solver': 'adam', 'max_iter': 500, 'learning_rate_init': 0.01, 'hidden_layer_sizes': 60, 'alpha': 0.01, 'activation': 'relu'}
best_params={'solver': 'lbfgs', 'max_iter': 200, 'learning_rate_init': 0.01, 'hidden_layer_sizes': 50, 'alpha': 0.001, 'activation': 'tanh'}
mlp = MLPClassifier(**best_params).fit(x_train,y_train)
mlp_p = mlp.predict(x_test)
print(accuracy_score(y_test,mlp_p))

0.9047619047619048


## Neural Network

In [73]:
from tensorflow import keras
from keras.models  import Sequential
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.optimizers import Adam, SGD, RMSprop

In [74]:
x_train.shape

(166, 60)

In [87]:
model = Sequential()

model.add(Flatten(input_shape=(60,)))

model.add(Dense(128,activation='tanh'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64,activation='tanh'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(32,activation='tanh'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(16,activation='tanh'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer = Adam() , loss = 'binary_crossentropy',metrics=['accuracy'])

model.fit(x_train,y_train,epochs = 100, batch_size= 32 , verbose ='auto',validation_split = 0.2)

Epoch 1/100
5/5 [==============================] - 8s 489ms/step - loss: 1.0777 - accuracy: 0.4167 - val_loss: 0.6980 - val_accuracy: 0.5882
Epoch 2/100
5/5 [==============================] - 0s 39ms/step - loss: 1.0063 - accuracy: 0.4773 - val_loss: 0.6829 - val_accuracy: 0.5588
Epoch 3/100
5/5 [==============================] - 0s 35ms/step - loss: 1.0422 - accuracy: 0.4848 - val_loss: 0.6761 - val_accuracy: 0.5588
Epoch 4/100
5/5 [==============================] - 0s 37ms/step - loss: 0.8373 - accuracy: 0.4924 - val_loss: 0.6726 - val_accuracy: 0.5588
Epoch 5/100
5/5 [==============================] - 0s 37ms/step - loss: 0.8742 - accuracy: 0.5379 - val_loss: 0.6681 - val_accuracy: 0.5588
Epoch 6/100
5/5 [==============================] - 0s 30ms/step - loss: 0.7443 - accuracy: 0.6136 - val_loss: 0.6632 - val_accuracy: 0.5588
Epoch 7/100
5/5 [==============================] - 0s 30ms/step - loss: 0.8722 - accuracy: 0.5455 - val_loss: 0.6572 - val_accuracy: 0.5588
Epoch 8/100
5/5 [==

In [88]:
model.evaluate(x_test,y_test)

2/2 [==============================] - 0s 8ms/step - loss: 0.4044 - accuracy: 0.8571


[0.4043585956096649, 0.8571428656578064]